In [1]:
# Load packages and check versions
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400 #high res figures
import matplotlib.pyplot as plt
%matplotlib inline
import graphviz
import sklearn
import xgboost as xgb
import shap

print('The Python version is {}.\n'.format(sys.version))
print('The Numpy version is {}.\n'.format(np.__version__))
print('The Pandas version is {}.\n'.format(pd.__version__))
print('The Matplotlib version is {}.\n'.format(mpl.__version__))
print('The Graphviz version is {}.\n'.format(graphviz.__version__))
print('The Scikit-Learn version is {}.\n'.format(sklearn.__version__))
print('The XGBoost version is {}.\n'.format(xgb.__version__))
print('The Shap version is {}.\n'.format(shap.__version__))
# The Python version is 3.8.2 (default, Mar 26 2020, 10:43:30) 
# [Clang 4.0.1 (tags/RELEASE_401/final)].

# The Numpy version is 1.19.2.

# The Pandas version is 1.2.1.

# The Matplotlib version is 3.3.2.

# The Graphviz version is 0.15.

# The Scikit-Learn version is 0.23.2.

# The XGBoost version is 1.3.0.

# The Shap version is 0.37.0.

The Python version is 3.8.2 (default, Mar 26 2020, 10:43:30) 
[Clang 4.0.1 (tags/RELEASE_401/final)].

The Numpy version is 1.19.2.

The Pandas version is 1.2.1.

The Matplotlib version is 3.3.2.

The Graphviz version is 0.15.

The Scikit-Learn version is 0.23.2.

The XGBoost version is 1.3.0.

The Shap version is 0.37.0.



Looked through a few datasets on the UCI repository and this one seems like a not-too-complicated and fairly business-relevant binary classification set (https://archive.ics.uci.edu/ml/datasets/Shill+Bidding+Dataset#).

In [2]:
df = pd.read_csv('../Data/Shill_Bidding_Dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Record_ID               6321 non-null   int64  
 1   Auction_ID              6321 non-null   int64  
 2   Bidder_ID               6321 non-null   object 
 3   Bidder_Tendency         6321 non-null   float64
 4   Bidding_Ratio           6321 non-null   float64
 5   Successive_Outbidding   6321 non-null   float64
 6   Last_Bidding            6321 non-null   float64
 7   Auction_Bids            6321 non-null   float64
 8   Starting_Price_Average  6321 non-null   float64
 9   Early_Bidding           6321 non-null   float64
 10  Winning_Ratio           6321 non-null   float64
 11  Auction_Duration        6321 non-null   int64  
 12  Class                   6321 non-null   int64  
dtypes: float64(8), int64(4), object(1)
memory usage: 642.1+ KB


- Record ID: Unique identifier of a record in the dataset.
- Auction ID: Unique identifier of an auction.
- Bidder ID: Unique identifier of a bidder.
- Bidder Tendency: A shill bidder participates exclusively in auctions of few sellers rather than a diversified lot. This is a collusive act involving the fraudulent seller and an accomplice.
- Bidding Ratio: A shill bidder participates more frequently to raise the auction price and attract higher bids from legitimate participants.
- Successive Outbidding: A shill bidder successively outbids himself even though he is the current winner to increase the price gradually with small consecutive increments.
- Last Bidding: A shill bidder becomes inactive at the last stage of the auction (more than 90\% of the auction duration) to avoid winning the auction.
- Auction Bids: Auctions with SB activities tend to have a much higher number of bids than the average of bids in concurrent auctions.
- Auction Starting Price: a shill bidder usually offers a small starting price to attract legitimate bidders into the auction.
- Early Bidding: A shill bidder tends to bid pretty early in the auction (less than 25\% of the auction duration) to get the attention of auction users.
- Winning Ratio: A shill bidder competes in many auctions but hardly wins any auctions.
- Auction Duration: How long an auction lasted.
- Class: 0 for normal behaviour bidding; 1 for otherwise.

In [4]:
df.head(20)

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.000000,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.000000,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.000000,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.000000,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.000000,0.000000,0.001242,0.500000,7,0
5,8,900,i***e,0.038462,0.111111,0.0,0.016844,0.000000,0.000000,0.016844,0.800000,7,0
6,10,900,m***p,0.400000,0.222222,0.0,0.006781,0.000000,0.000000,0.006774,0.750000,7,0
7,12,900,k***a,0.137931,0.444444,1.0,0.768044,0.000000,0.000000,0.016311,1.000000,7,1
8,13,2370,g***r,0.121951,0.185185,1.0,0.035021,0.333333,0.993528,0.023963,0.944444,7,1
9,27,600,e***t,0.155172,0.346154,0.5,0.570994,0.307692,0.993593,0.413788,0.611111,7,1


In [5]:
df.nunique()

Record_ID                 6321
Auction_ID                 807
Bidder_ID                 1054
Bidder_Tendency            489
Bidding_Ratio              400
Successive_Outbidding        3
Last_Bidding              5807
Auction_Bids                49
Starting_Price_Average      22
Early_Bidding             5690
Winning_Ratio               72
Auction_Duration             5
Class                        2
dtype: int64

In [6]:
df.describe()

,Record_ID,Auction_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
count,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000
mean,7535.829457,1241.388230,0.142541,0.127670,0.103781,0.463119,0.231606,0.472821,0.430683,0.367731,4.615093,0.106787
std,4364.759137,735.770789,0.197084,0.131530,0.279698,0.380097,0.255252,0.489912,0.380785,0.436573,2.466629,0.308867
min,1.000000,5.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,3778.000000,589.000000,0.027027,0.043478,0.000000,0.047928,0.000000,0.000000,0.026620,0.000000,3.000000,0.000000
50%,7591.000000,1246.000000,0.062500,0.083333,0.000000,0.440937,0.142857,0.000000,0.360104,0.000000,5.000000,0.000000
75%,11277.000000,1867.000000,0.166667,0.166667,0.000000,0.860363,0.454545,0.993593,0.826761,0.851852,7.000000,0.000000
max,15144.000000,2538.000000,1.000000,1.000000,1.000000,0.999900,0.788235,0.999935,0.999900,1.000000,10.000000,1.000000


Potentially a complicated EDA here and some of these feature values are not clearly described. As opposed to getting too far down in to the weeds here, I should probably just have some kind of hand-wavy statement about what the feature values are intended to represent and then just get down to using them.

One interesting thing here is the lack of time data; if a bidder has been previously identified as a shill bidder, won't that make them more likely to do it in the future?

How did they even get the response variable for this? I suppose I could be reading papers and learning, and maybe I'll skim, but this could potentially take a lot of time.

---

How to do data splitting here? Maybe just have train and validation, since we won't be using this later.

# ***Start here

In [7]:
 features = df.columns.tolist()

In [8]:
features

['Record_ID',
 'Auction_ID',
 'Bidder_ID',
 'Bidder_Tendency',
 'Bidding_Ratio',
 'Successive_Outbidding',
 'Last_Bidding',
 'Auction_Bids',
 'Starting_Price_Average',
 'Early_Bidding',
 'Winning_Ratio',
 'Auction_Duration',
 'Class']

In [9]:
for item in ['Record_ID', 'Auction_ID', 'Bidder_ID', 'Class']:
    features.remove(item)

In [10]:
features

['Bidder_Tendency',
 'Bidding_Ratio',
 'Successive_Outbidding',
 'Last_Bidding',
 'Auction_Bids',
 'Starting_Price_Average',
 'Early_Bidding',
 'Winning_Ratio',
 'Auction_Duration']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, y_train, y_val = \
train_test_split(df[features].values,
                 df['Class'].values,
                 test_size=0.2, random_state=24)

In [16]:
xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.3,
    verbosity=1,
    objective='binary:logistic',
    use_label_encoder=False,
    n_jobs=-1,
    min_child_weight=100)
params = {'max_depth':[6],
          'n_estimators':[1000],
          'colsample_bytree':[0.9],
          'subsample':[0.9]}
#Above can beat the RF in the 4-fold CV
# 0.777226

# xgb_model = xgb.XGBClassifier(
#     n_estimators=1000,
#     max_depth=0,
#     max_leaves=100,
#     learning_rate=0.01,
#     verbosity=1,
#     objective='binary:logistic',
#     use_label_encoder=False,
#     n_jobs=-1,
#     min_child_weight=100,
#     tree_method='hist',
#     grow_policy='lossguide')
# params = {'n_estimators':[1000],
#           'colsample_bytree':[0.9],
#           'subsample':[0.9]}
# # 0.777415 so comparable, and works a lot faster than the above



In [17]:
eval_set = [(X_train, y_train), (X_val, y_val)]

In [18]:
%%time
xgb_model.fit(X_train, y_train, eval_set=eval_set,
          eval_metric="auc", verbose=True, early_stopping_rounds=30)

[0]	validation_0-auc:0.98276	validation_1-auc:0.98574
[1]	validation_0-auc:0.98276	validation_1-auc:0.98574
[2]	validation_0-auc:0.98276	validation_1-auc:0.98574
[3]	validation_0-auc:0.98276	validation_1-auc:0.98574
[4]	validation_0-auc:0.98276	validation_1-auc:0.98574
[5]	validation_0-auc:0.98276	validation_1-auc:0.98574
[6]	validation_0-auc:0.98276	validation_1-auc:0.98574
[7]	validation_0-auc:0.98925	validation_1-auc:0.99198
[8]	validation_0-auc:0.98925	validation_1-auc:0.99198
[9]	validation_0-auc:0.98925	validation_1-auc:0.99198
[10]	validation_0-auc:0.98925	validation_1-auc:0.99198
[11]	validation_0-auc:0.98898	validation_1-auc:0.99299
[12]	validation_0-auc:0.98898	validation_1-auc:0.99299
[13]	validation_0-auc:0.98898	validation_1-auc:0.99299
[14]	validation_0-auc:0.98898	validation_1-auc:0.99299
[15]	validation_0-auc:0.98898	validation_1-auc:0.99299
[16]	validation_0-auc:0.98898	validation_1-auc:0.99299
[17]	validation_0-auc:0.98898	validation_1-auc:0.99299
[18]	validation_0-au

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=3,
              min_child_weight=100, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=1)

In [11]:
%time cv_xgb.fit(X_train, y_train)

Fitting 4 folds for each of 1 candidates, totalling 4 fits
[13:34:13] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[13:34:18] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:34:22] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:34:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   17.2s finished


[13:34:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CPU times: user 1min 12s, sys: 2.1 s, total: 1min 14s
Wall time: 22.5 s


GridSearchCV(cv=4,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, grow_policy='lossguide',
                                     importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.01, max_delta_step=None,
                                     max_depth=0, max_leaves=100,
                                     min_child_weight=100, missing=nan,
                                     monotone_const...
                                     n_estimators=1000, n_jobs=-1,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                             

In [12]:
xgb_cv_results_df = pd.DataFrame(cv_xgb.cv_results_)

In [13]:
xgb_cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
0,4.026787,0.111627,0.071355,0.001617,0.9,1000,0.9,"{'colsample_bytree': 0.9, 'n_estimators': 1000...",0.771406,0.769602,...,0.778443,0.777415,0.008091,1,0.824991,0.827054,0.82156,0.825088,0.824673,0.001977


Shaps

In [14]:
%time explainer = shap.TreeExplainer(cv_xgb)

Exception: Model type not yet supported by TreeExplainer: <class 'sklearn.model_selection._search.GridSearchCV'>